In [2]:
# imports:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import numpy as np
import random
import sklearn
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
%matplotlib inline
import xlrd 
import csv
import tensorflow as tf
import seaborn as sns
import plotly.express as px
from pandas.api.types import CategoricalDtype
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
df = pd.read_csv("sign_mnist_test.csv")

In [ ]:
# mean, median, mode

In [10]:
print("\n----------- Calculate Mean -----------\n")
print(df.mean())
 
print("\n----------- Calculate Median -----------\n")
print(df.median())
 
print("\n----------- Calculate Mode -----------\n")
print(df.mode())


----------- Calculate Mean -----------

label        11.247351
pixel1      147.532627
pixel2      150.445761
pixel3      153.324317
pixel4      155.663413
               ...    
pixel780    159.648494
pixel781    158.162019
pixel782    157.672755
pixel783    156.664250
pixel784    154.776771
Length: 785, dtype: float64

----------- Calculate Median -----------

label        11.0
pixel1      154.0
pixel2      157.0
pixel3      159.0
pixel4      161.0
            ...  
pixel780    182.0
pixel781    181.0
pixel782    180.0
pixel783    180.0
pixel784    179.0
Length: 785, dtype: float64

----------- Calculate Mode -----------

   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0    4.0   171.0   158.0   180.0   168.0   173.0   174.0   179.0   171.0   
1    NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2    NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel77

In [ ]:
#variance

In [11]:
df.var()

label         55.453521
pixel1      1900.362204
pixel2      1752.915898
pixel3      1635.614229
pixel4      1548.798421
               ...     
pixel780    4290.167181
pixel781    4421.395584
pixel782    4357.279174
pixel783    4516.235046
pixel784    4662.861471
Length: 785, dtype: float64

In [ ]:
# standard deviation 

In [8]:
df.std() 

label        7.446712
pixel1      43.593144
pixel2      41.867838
pixel3      40.442728
pixel4      39.354776
              ...    
pixel780    65.499368
pixel781    66.493576
pixel782    66.009690
pixel783    67.202939
pixel784    68.285148
Length: 785, dtype: float64

In [ ]:
# inter quartile range - IQR


In [16]:
df.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,...,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000,7172.000000
mean,11.247351,147.532627,150.445761,153.324317,155.663413,158.169688,160.790853,162.282766,163.649191,165.589515,...,138.546570,145.539598,150.744980,155.638873,158.893196,159.648494,158.162019,157.672755,156.664250,154.776771
std,7.446712,43.593144,41.867838,40.442728,39.354776,37.749637,36.090916,36.212636,35.885378,33.721876,...,64.501665,65.132370,65.760539,65.565147,65.200300,65.499368,66.493576,66.009690,67.202939,68.285148
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,122.000000,126.000000,130.000000,134.000000,137.000000,141.000000,144.000000,145.000000,147.000000,...,90.000000,95.000000,99.000000,105.000000,113.000000,113.750000,113.000000,115.000000,111.000000,106.750000
50%,11.000000,154.000000,157.000000,159.000000,161.000000,163.000000,165.000000,166.000000,168.000000,169.000000,...,137.000000,155.000000,168.000000,177.000000,181.000000,182.000000,181.000000,180.000000,180.000000,179.000000
75%,18.000000,178.000000,179.000000,181.000000,182.000000,184.000000,185.000000,186.000000,187.000000,187.000000,...,195.000000,200.000000,204.250000,207.000000,207.000000,208.000000,207.000000,205.000000,206.000000,204.000000
max,24.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000


In [ ]:
# To find out and filter such outliers in the dataset we will create a custom function that 
# will help us remove outliers. In the function, we first need to find out the IQR value that 
# can be calculated by finding the difference between the third and first quartile values. 
# Secondly, we will write a query to select observations that lie outside the lower_range and 
# upper_range IQR region and remove them.

In [22]:
# Removing the outliers
def removeOutliers(df, col):
    Q3 = np.quantile(df[col], 0.75)
    Q1 = np.quantile(df[col], 0.25)
    IQR = Q3 - Q1
 
    print("IQR value for column %s is: %s" % (col, IQR))
    global outlier_free_list
    global filtered_data
 
    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    outlier_free_list = [x for x in df[col] if (
        (x > lower_range) & (x < upper_range))]
    filtered_data = df.loc[df[col].isin(outlier_free_list)]
 
 
for i in df.columns:
    if i == df.columns[0]:
      removeOutliers(df, i)
    else:
      removeOutliers(filtered_data, i)
 
  
# Assigning filtered data back to our original variable
df = filtered_data
print("Shape of data after outlier removal is: ", df.shape)

IQR value for column label is: 14.0
IQR value for column pixel1 is: 56.0
IQR value for column pixel2 is: 52.0
IQR value for column pixel3 is: 48.0
IQR value for column pixel4 is: 46.0
IQR value for column pixel5 is: 44.0
IQR value for column pixel6 is: 42.0
IQR value for column pixel7 is: 40.0
IQR value for column pixel8 is: 38.0
IQR value for column pixel9 is: 37.0
IQR value for column pixel10 is: 35.0
IQR value for column pixel11 is: 35.0
IQR value for column pixel12 is: 34.0
IQR value for column pixel13 is: 34.0
IQR value for column pixel14 is: 34.0
IQR value for column pixel15 is: 33.0
IQR value for column pixel16 is: 32.0
IQR value for column pixel17 is: 32.0
IQR value for column pixel18 is: 32.0
IQR value for column pixel19 is: 32.0
IQR value for column pixel20 is: 32.0
IQR value for column pixel21 is: 31.0
IQR value for column pixel22 is: 31.0
IQR value for column pixel23 is: 31.0
IQR value for column pixel24 is: 32.0
IQR value for column pixel25 is: 32.0
IQR value for column pi

IQR value for column pixel225 is: 41.0
IQR value for column pixel226 is: 39.25
IQR value for column pixel227 is: 37.0
IQR value for column pixel228 is: 32.0
IQR value for column pixel229 is: 30.0
IQR value for column pixel230 is: 32.0
IQR value for column pixel231 is: 30.0
IQR value for column pixel232 is: 31.0
IQR value for column pixel233 is: 30.0
IQR value for column pixel234 is: 35.0
IQR value for column pixel235 is: 36.0
IQR value for column pixel236 is: 40.0
IQR value for column pixel237 is: 46.0
IQR value for column pixel238 is: 48.0
IQR value for column pixel239 is: 48.0
IQR value for column pixel240 is: 55.0
IQR value for column pixel241 is: 56.0
IQR value for column pixel242 is: 57.0
IQR value for column pixel243 is: 58.0
IQR value for column pixel244 is: 52.0
IQR value for column pixel245 is: 55.0
IQR value for column pixel246 is: 49.0
IQR value for column pixel247 is: 45.5
IQR value for column pixel248 is: 33.0
IQR value for column pixel249 is: 30.0
IQR value for column pix

IQR value for column pixel488 is: 35.0
IQR value for column pixel489 is: 30.75
IQR value for column pixel490 is: 38.25
IQR value for column pixel491 is: 36.5
IQR value for column pixel492 is: 54.75
IQR value for column pixel493 is: 92.75
IQR value for column pixel494 is: 79.25
IQR value for column pixel495 is: 63.0
IQR value for column pixel496 is: 79.75
IQR value for column pixel497 is: 72.75
IQR value for column pixel498 is: 32.0
IQR value for column pixel499 is: 22.0
IQR value for column pixel500 is: 23.0
IQR value for column pixel501 is: 22.0
IQR value for column pixel502 is: 20.5
IQR value for column pixel503 is: 22.0
IQR value for column pixel504 is: 21.0
IQR value for column pixel505 is: 21.0
IQR value for column pixel506 is: 19.5
IQR value for column pixel507 is: 19.5
IQR value for column pixel508 is: 16.5
IQR value for column pixel509 is: 15.0
IQR value for column pixel510 is: 16.0
IQR value for column pixel511 is: 17.0
IQR value for column pixel512 is: 16.25
IQR value for col

IQR value for column pixel775 is: 4.5
IQR value for column pixel776 is: 5.5
IQR value for column pixel777 is: 7.0
IQR value for column pixel778 is: 1.0
IQR value for column pixel779 is: 1.0
IQR value for column pixel780 is: 3.0
IQR value for column pixel781 is: 5.0
IQR value for column pixel782 is: 6.0
IQR value for column pixel783 is: 2.0
IQR value for column pixel784 is: 1.0
Shape of data after outlier removal is:  (5, 785)


In [24]:
# skewness
df.skew(axis = 0, skipna = True)

label       0.000000
pixel1     -0.608581
pixel2      0.000000
pixel3      0.608581
pixel4      0.000000
              ...   
pixel780    0.912717
pixel781   -0.088041
pixel782   -1.016267
pixel783   -0.518421
pixel784   -0.608581
Length: 785, dtype: float64

In [25]:
df.skew(axis = 1, skipna = True)

2428   -1.322531
3236   -1.315081
3278   -1.322861
4403   -1.318324
4718   -1.316462
dtype: float64